# Google Colaboratory を利用した ieinn ライブラリの使用例
Google Colaboratory(以下 Google Colab)を利用すれば、簡単に Python の実行環境を構築できます. GPU も無料で使用できます. ここでは Google Colab を利用した ieinn ライブラリの使用例を紹介します. 
## ieinnライブラリを環境に複製し、移動します。

In [ ]:
!git clone https://github.com/AoiHonda-lab/IEI-NeuralNetwork.git

In [ ]:
%cd IEI-NeuralNetwork

## 必要なライブラリをインポートします

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import csv
from sklearn.model_selection import train_test_split
from ieinn import ieinn

GPUが使える場合は deviceに cuda を設定、そうでなければ cpu を設定します。

In [1]:
#device = 'cuda' if torch.cuda.is_available() else 'cpu'
device ='cpu'
print(device)

cpu


## データの読み込み
データの読み込みと、訓練データと検証データへの分割などの処理をします。

In [ ]:
df=pd.read_csv('CarEvaluation20221207.csv',encoding="shift-jis")
df=df.drop(0,axis=0)
df=df.astype(float)

y=pd.DataFrame(df.iloc[:,0])
X=df.drop(df.columns[[0]], axis=1)

#Normalize the objective variable
y=(y - y.min()) / (y.max() - y.min())

# data Generating
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_df=X_train
y_train_df=y_train

# Extract as a numpy array with value and convert to tensor
X_train = torch.FloatTensor(X_train.values)
y_train = torch.FloatTensor(y_train.values)
X_test = torch.FloatTensor(X_test.values)
y_test = torch.FloatTensor(y_test.values)

In [ ]:
print(df)

## データセット、データローダを作成
訓練データ、テストデータでそれぞれデータセットを作成し、これらを引数としてデータローダーを作成します。ミニバッチサイズやシャッフルの指定をします。

In [ ]:
# Dataset creating
train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
test_dataset = torch.utils.data.TensorDataset(X_test, y_test)

# DataLoade creating
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=64, shuffle=True, num_workers=2, generator=torch.Generator(device=device))
#train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=128, shuffle=True, num_workers=2)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=64, shuffle=False, num_workers=2, generator=torch.Generator(device=device))
#test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=128, shuffle=False, num_workers=2)

## 学習モデルの作成
インポートした ieinn の IE クラスでモデルを作成します。引数として、訓練データローダ－、ファジィ測度の加法性の次数、積分に用いる多項演算、前処理の種類を渡します。
訓練データローダは、上で作成した train_loader、加法性の次数は1から説明変数の数までの整数を渡します。多項演算は、論理積:Min、代数積:Algebraic、限界積:Lukasiewiczなどのt-norm が準備されています。前処理は、PreprocessingLayerPercentile, PreprocessingLayerStandardDeviation, PreprocessingLayerMaxMin から選べます。加法性次数、多項演算、前処理手法を指定しない場合、デフォルトはそれぞれ、完全非加法、代数積、PreprocessingLayerPercentile となっています。

In [ ]:
model=ieinn.IE(train_loader,additivity_order=4,narray_op='Min').to(device)
criterion = nn.MSELoss() #loss function
optimizer = torch.optim.Adam(model.parameters()) #Optimization method
print(model)

## 学習前のパラメータを確認

In [ ]:
model.state_dict()

## 学習の実行

In [ ]:
if (device == 'cuda'):
  torch.set_default_tensor_type('torch.cuda.FloatTensor')
history=model.fit_and_valid(train_loader, test_loader, criterion, optimizer, epochs=100, device=device)

## 学習結果の表示
学習後の決定係数、パラメータ、学習曲線を表示します。

In [ ]:
model.r2_score(test_loader,device)

In [ ]:
model.state_dict()

In [ ]:
model.plot()

In [ ]:
model.plot_train()

In [ ]:
model.plot_test()